<a href="https://colab.research.google.com/github/Monica0806/emotion-movie-recommender/blob/main/Movie_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [9]:

!pip install transformers
import pandas as pd
from transformers import pipeline

movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
tags = pd.read_csv('/content/tags.csv')

print("Movies:")
print(movies.head())

print("\nRatings:")
print(ratings.head())

print("\nTags:")
print(tags.head())


Movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings:
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976
3       1       30     5.0  944249077
4       1       32     5.0  943228858

Tags:
   userId  movieId          tag   timestamp
0      22    26479  Kevin Kline  1583038886
1      22    79592     misogyny  1581476297
2      22   247150  

Emotion Detection Pipeline

In [10]:
emotion_model = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    return_all_scores=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Test Emotion Detection

In [11]:
def detect_emotion(text):
    result = emotion_model(text)
    return result[0]['label']

In [13]:
mood = "I feel really low and heartbroken"
emotion = detect_emotion(mood)
print(f"\nDetected Emotion: {emotion}")


Detected Emotion: sadness


Map Emotions to Genres

In [14]:
emotion_to_genres = {
    "sadness": ["Drama", "Romance"],
    "joy": ["Comedy", "Adventure"],
    "love": ["Romance", "Family"],
    "anger": ["Action", "Crime"],
    "fear": ["Horror", "Thriller"],
    "surprise": ["Mystery", "Fantasy"]
}

In [15]:
def get_movies_by_emotion(emotion):
    genres = emotion_to_genres.get(emotion.lower(), [])
    if not genres:
        return pd.DataFrame()
    pattern = '|'.join(genres)
    return movies[movies['genres'].str.contains(pattern, case=False, na=False)]

Recommend Top Movies

In [16]:
avg_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
avg_ratings.columns = ['movieId', 'avg_rating']

def recommend_movies_by_emotion(user_input, top_n=10):
    emotion = detect_emotion(user_input)
    emotion_movies = get_movies_by_emotion(emotion)
    if emotion_movies.empty:
        return f"No movies found for emotion: {emotion}"

    merged = pd.merge(emotion_movies, avg_ratings, on='movieId')
    top_movies = merged.sort_values(by='avg_rating', ascending=False).head(top_n)
    return emotion, top_movies[['title', 'avg_rating']]

In [20]:
user_feeling = input("How are you feeling today?: ")
emotion, recommended = recommend_movies_by_emotion(user_feeling)

print(f"\n🎯 Based on your mood, we recommend:\n")
for idx, row in recommended.iterrows():
    print(f"🎬 {row['title']} — ⭐ {round(row['avg_rating'], 2)}")

How are you feeling today?: happy

🎯 Based on your mood, we recommend:

🎬 Retreat to You (2023) — ⭐ 5.0
🎬 Heritage Falls (2016) — ⭐ 5.0
🎬 Portugal... Minha Saudade (1974) — ⭐ 5.0
🎬 Um Caipira em Bariloche (1973) — ⭐ 5.0
🎬 Jeca Contra o Capeta (1976) — ⭐ 5.0
🎬 Jecão... Um Fofoqueiro no Céu (1977) — ⭐ 5.0
🎬 Jeca e Seu Filho Preto (1978) — ⭐ 5.0
🎬 A Banda das Velhas Virgens (1979) — ⭐ 5.0
🎬 High and Hallowed: Everest 1963 (2013) — ⭐ 5.0
🎬 A Feeling of Home (2019) — ⭐ 5.0
